In [3]:
import pandas as p
import numpy as np
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk import pos_tag
from nltk.util import ngrams
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix as cm

from sklearn.cross_validation import train_test_split

In [4]:
train = p.read_csv('./sub_obj_data/train_new.csv', usecols=(['class', 'text'])).dropna()
# test  = p.read_csv('./sub_obj_data/test_ds.csv', usecols=(['class', 'text'])).dropna()
# train = train.reindex(np.random.permutation(train.index))
training_data, test_data, training_labels, test_labels = train_test_split(train['text'].values, train['class'].values, test_size=0.3, random_state=0) #70-30 split
print len(training_data), len(test_data)

4162 1784


In [5]:

import scipy.sparse as sp
def build_and_evaluate(X, y, X_test, y_test, outpath=None):

    def preprocess(s):
        return preprocessor.tokenise(s)

    clf = LogisticRegression(penalty='l2')
    preprocessor = Preprocessor(X)
    vec = TfidfVectorizer(tokenizer=preprocess,
                          lowercase=False,
                          ngram_range=(1, 1),
                          max_features=5000)
    

    # Build model
    print("Building model")
    matrix = vec.fit_transform(X)
    clf.fit(matrix, y)

    # Evaluate on test set
    matrix = vec.transform(X_test)
    y_pred = clf.predict(matrix)

    print("Classification Report:\n")
    print np.mean(y_pred == y_test)
    print cm(y_test, y_pred)
    print(clsr(y_test, y_pred, target_names=['obj', 'sub']))

In [12]:
from __future__ import unicode_literals, division
import re
import htmlentitydefs
import csv

from nltk.corpus import stopwords as sw
from nltk import pos_tag


class Preprocessor():

    def __init__(self, corpus):
        self.stopwords = sw.words('english')
        self.word_re = word_re
        self.emoticon_re = emoticon_re
        self.html_entity_digit_re = html_entity_digit_re
        self.html_entity_alpha_re = html_entity_alpha_re
        self.amp = amp
        self.punct_re = punct_re
        self.negation_re = negation_re
        self.url_re = url_re
        self.rep_char_re = rep_char_re
        self.hashtag_re = hashtag_re
        self.user_tag_re = user_tag_re
        self.acrynoms = self.load_acrynoms()
        self.stemmer = PorterStemmer()
        self.top_bigrams = self.get_bigrams(corpus)

    def load_acrynoms(self):
        with open('./data/acrynom.csv', 'rb') as f:
            reader = csv.reader(f)
            slang = dict((rows[0], rows[1]) for rows in reader)
            return slang
    
    def helper(self, tweet):
        tweet = self.__html2unicode(tweet)
        tokens = self.word_re.findall(tweet)
        vect = []
        for t in tokens:
            if t in self.stopwords or t in string.punctuation:
                continue
            vect.append(t)
        
        return vect
        
    def get_bigrams(self, corpus):
        tokens = []
        for tweet in corpus:
            tokens.append(self.helper(tweet.decode('utf-8').lower()))
        tokens = [val for sublist in tokens for val in sublist]
        
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        finder = BigramCollocationFinder.from_words(tokens)
        finder.apply_freq_filter(5)
        return finder.nbest(bigram_measures.likelihood_ratio, 1000)
    
    def pos_tags(self, tokens):
        TAG_MAP = [ "NN", "NNP", "NNS", "VBP", "VB", "VBD", 'VBG', "VBN",
                    "VBZ", "MD","UH", "PRP", "PRP$"]
        tags = pos_tag(tokens)
        return [tag[1] for tag in tags if tag[1] in TAG_MAP]

    def normalise(self, tokens):
        vect = []
        
        bigrams = ngrams(tokens, 2)
        
        for bigram in bigrams:
            bigram_lower = (bigram[0].lower(), bigram[1].lower())
            if bigram_lower in self.top_bigrams:
                if bigram_lower[0] not in self.stopwords or bigram_lower[1] not in self.stopwords:
                    vect.append(bigram_lower[0] + '_' + bigram_lower[1])

        for t in tokens:

            if not self.emoticon_re.search(t):
                t = t.lower()
            if t in string.punctuation or t in self.stopwords:
                continue     
            
            t = self.rep_char_re.sub(r'\1', t)
            t = self.url_re.sub('_URL', t)
            t = self.hashtag_re.sub('_HASH', t)
            t = self.user_tag_re.sub('_USER', t)
            
            vect.append(self.stemmer.stem(t))
#             vect.append(t)
        
        tags = self.pos_tags(vect)
        vect = tags + vect
        return vect

    def tokenise(self, tweet):
        tweet = self.__html2unicode(tweet)
        tokens = self.word_re.findall(tweet)
#         tknzr = TweetTokenizer(reduce_len=True)
#         tokens = tknzr.tokenize(tweet)
        return self.normalise(tokens)

    def ensure_unicode(self, tweet):
        try:
            return unicode(tweet)
        except UnicodeDecodeError:
            tweet = str(tweet).encode('string_escape')
            return unicode(tweet)

    def __html2unicode(self, s):
        """
        This function is curtosy of Christopher Potts
        http://sentiment.christopherpotts.net/index.html
        Internal metod that seeks to replace all the HTML entities in
        s with their corresponding unicode characters.
        """
        # First the digits:
        ents = set(self.html_entity_digit_re.findall(s))
        if len(ents) > 0:
            for ent in ents:
                entnum = ent[2:-1]
                try:
                    entnum = int(entnum)
                    s = s.replace(ent, unichr(entnum))
                except:
                    pass
        # Now the alpha versions:
        ents = set(self.html_entity_alpha_re.findall(s))
        ents = filter((lambda x: x != amp), ents)
        for ent in ents:
            entname = ent[1:-1]
            try:
                s = s.replace(ent,
                              unichr(htmlentitydefs.name2codepoint[entname]))
            except:
                pass
            s = s.replace(self.amp, " and ")
        return s

    
"""
    This file is based on the work of Christopher Potts
    however the file has been altered and extended for
    my purposes
    http://sentiment.christopherpotts.net/index.html
"""
emoticon_string = r"""
    (?:
      [<>]?
      [:;=8]                     # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8]                     # eyes
      [<>]?
    )"""

# The components of the tokenizer:
regex_strings = (
    # Phone numbers:
    r""""
    (?:
      (?:            # (international)
        \+?[01]
        [\-\s.]*
      )?
      (?:            # (area code)
        [\(]?
        \d{3}
        [\-\s.\)]*
      )?
      \d{3}          # exchange
      [\-\s.]*
      \d{4}          # base
    )""",
    # Emoticons:
    emoticon_string,
    # HTML tags:
    r'<[^>]+>',
    # Twitter username:
    r'(?:@[\w_]+)',
    # Links
    r'http\S+',
    # Twitter hashtags:
    r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)',
    # Remaining word types:
    r"""
    (?:[a-z][a-z'\-_]+[a-z])       # Words with apostrophes or dashes.
    |
    (?:[+\-]?\d+[,/.:-]\d+[+\-]?)  # Numbers, including fractions, decimals.
    |
    (?:[\w_]+)                    # Words without apostrophes or dashes.
    |
    (?:\.(?:\s*\.){1,})            # Ellipsis dots.
    |
    (?:\S)                         # Everything else that isn't whitespace
    """
    )

negation_words = (
    """
    (?x)(?:
    ^(?:never|no|nothing|nowhere|noone|none|not|
        havent|hasnt|hadnt|cant|couldnt|shouldnt|
        wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint
     )$
    )
    |
    n't
    """
    )

# ######################################################################

word_re = re.compile(r'(%s)' % "|".join(regex_strings), re.VERBOSE | re.I | re.UNICODE)
emoticon_re = re.compile(regex_strings[1], re.VERBOSE | re.I | re.UNICODE)
html_entity_digit_re = re.compile(r'&#\d+;')
html_entity_alpha_re = re.compile(r'&\w+;')
amp = "&amp;"
punct_re = re.compile("^[.:;!?]$")
negation_re = re.compile(negation_words)
url_re = re.compile(r'http\S+')
rep_char_re = re.compile(r'(\w)\1{3,}')
hashtag_re = re.compile(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)')
user_tag_re = re.compile(r'(?:@[\w_]+)')


## remove punct from pos_tags

In [31]:
to = Preprocessor(train['text'])
l = []
for i, row in train.iterrows():
    l.append(to.tokenise(row['text'].decode('utf-8')))

In [32]:
print l

[['NN', 'NN', 'VBN', 'NN', u'_USER', u'yeah', u'great', u'vid', u'12', u'singl', u'sold', u'year', u'ago'], ['NN', 'NN', 'VB', 'NN', 'NN', 'VBP', 'NN', 'PRP', 'MD', 'VB', 'NN', u'_USER', u"i'll", u'get', u'stuff', u'fr', u'uk', u'japan', u"i'm", u'desper', u'enough', u'headphon', u'us', u'might', u'jac', u'tita', u'slow'], ['NN', 'NN', 'NNS', 'VBP', 'NN', u'_USER', u"it'", u'never', u'late', u'thank', u'hard', u'hard', u'concentr', u'take', u'screencap'], ['NN', 'NN', u"c'mon", u'sean', u'man'], ['NN', 'NN', 'VB', 'NN', 'NN', 'NN', u'bed', u'...', u'suddenli', u'feel', u'wish', u'w', u'...', u'goodnight', u'twitterfam'], ['NN', 'NN', 'NN', 'NN', 'NN', 'VBN', 'NN', u'feb', u'issu', u'splurg', u'right', u'lent', u'nova', u'splurg', u'given', u'lent'], ['NN', 'NN', 'NN', 'NN', 'VB', u'ricki', u'pont', u'pass', u'23,000', u'first', u'class', u'run', u'12th', u'australian'], ['NN', 'VBP', 'NN', 'VBD', 'NN', 'NN', 'VB', 'NNS', 'NNS', u'_USER', u'go', u'phuk', u'citi', u'sept', u'15', u'take'

In [11]:
model = build_and_evaluate(training_data, training_labels, test_data, test_labels)

Building model
Classification Report:

0.901905829596
[[831  56]
 [119 778]]
             precision    recall  f1-score   support

        obj       0.87      0.94      0.90       887
        sub       0.93      0.87      0.90       897

avg / total       0.90      0.90      0.90      1784



In [13]:
model = build_and_evaluate(training_data, training_labels, test_data, test_labels)

Building model
Classification Report:

0.903587443946
[[833  54]
 [118 779]]
             precision    recall  f1-score   support

        obj       0.88      0.94      0.91       887
        sub       0.94      0.87      0.90       897

avg / total       0.91      0.90      0.90      1784



In [51]:
from nltk.tokenize import TweetTokenizer
stopwords = sw.words('english')
tknzr = TweetTokenizer(reduce_len=True)
s0 = "@hell 2342,00 This is a cooool #dummysmiley!!!! it's : :-) :-P <3 and some arrows < > -> <--"
tokens = tknzr.tokenize(s0)

vect = []
for t in tokens:
    if not emoticon_re.search(t):
        t = t.lower()
    if t in string.punctuation or t in stopwords:
        continue     

    t = rep_char_re.sub(r'\1', t)
    t = url_re.sub('_URL', t)
    t = hashtag_re.sub('_HASH', t)
    t = user_tag_re.sub('_USER', t)

    vect.append(t)

print vect

[u'_USER', u'2342,00', u'coool', u'_HASH', u"it's", u':-)', u':-P', u'<3', u'arrows', u'->', u'<--']
